<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 10 - Definición del modelo LightFM y entrenamiento</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Definición de funcion para entrenamiento](#section1)
* [2. Carga de datos](#section2)
* [3. Proceso de entrenamiento](#section3)

<br>

---

In [ ]:
# Permite establecer la anchura de la celda
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import joblib
import numpy as np
import os
import pandas as pd
import pickle
import time

from lightfm import LightFM
from modules.TelegramBot import telegram_bot_sendtext

In [ ]:
# Variables globales
SEED = 1
NUM_THREADS = 8

# Directorio empleado para guardar/leer los datos generados
MPD_CSV_PATH = 'MPD_CSV'
MODELS_PATH = 'models'
MPD_LFM_PATH = os.path.join(MPD_CSV_PATH, "mpd_lightfm.pickle")

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Definición de funcion para entrenamiento</font>
<br>


<br>

In [ ]:
# Función que se encarga de entrenar el modelo y una vez que ha concluido el proceso
# lo almacena en un fichero mediante joblib.
def train_model(model, interactions, interactions_weights=None, item_features=None, user_features=None,
                model_name='model', models_path='', n_epochs=10, n_threads=1, verbose=True, process_name=''):
    train_error = False

    ## Entrenamiento ##
    try:
        print("Comienza el entrenamiento")
        telegram_bot_sendtext("Comienza el entrenamiento",process_name)

        start_time = time.time()
        model.fit(interactions=interactions, sample_weight=interactions_weights,
                  item_features=item_features, user_features=user_features,
                  epochs=n_epochs, num_threads=n_threads, verbose=True)
        duration = (time.time() - start_time)

        duration = np.round(duration/60,2)
        print("Entrenamiento completado. Tiempo empleado: {} min.".format(duration))
        telegram_bot_sendtext("Entrenamiento completado. Tiempo empleado {} min.".format(duration), process_name)          
    except Exception as e:
        print(e)
        train_error = True
        telegram_bot_sendtext("Error: se ha producido un error durante el proceso de entrenamiento",process_name)       

    ## Almacenamiento del modelo ##
    try:
        if not train_error:
            print("Almacenando modelo ...")
            telegram_bot_sendtext("Almacenando modelo",process_name)
            model_file_path = os.path.join(models_path,"{}.pkl".format(model_name))
            joblib.dump(model, open(model_file_path, 'wb'))
    except Exception as e:
        print(e)
        train_error = True
        telegram_bot_sendtext("Error: el modelo no ha podido ser almacenado", process_name)

    ## Comprobación del modelo almacenado ##
    try:
        if not train_error:
            joblib.load(open(model_file_path, 'rb'))
            print('Modelo almacendo correctamente')
            telegram_bot_sendtext('Modelo almacenado correctamente',process_name)
    except:
        print(e)
        telegram_bot_sendtext("Error: el modelo no se ha almacenado correctamente", process_name)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Carga de datos</font>

<br>

Para el entrenamiento del modelo, únicamente es necesario emplear las matrices creamos en el preprocesamiento y que se encuentran almacenadas en un diccionario dentro de un fichero de *pickle*.

In [ ]:
with open(MPD_LFM_PATH, "rb") as read_file:
    mpd_lfm_dict = pickle.load(read_file)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Proceso de entrenamiento</font>

<br>

Una vez que hemos creado la función que realizará el entrenamiento y guardará el modelo resultante, comenzamos con el entrenamiento:

In [ ]:
model_plstrs = LightFM(loss='warp', no_components=200, max_sampled=30, random_state=SEED)

In [ ]:
train_model(model_plstrs, model_name='tfg_model',n_epochs=150, n_threads=NUM_THREADS, models_path=MODELS_PATH,
            interactions=mpd_lfm_dict['plstrs_interactions'], user_features=mpd_lfm_dict['pls_features'])

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>